In [1]:
import sys
import os
import enum 
import asyncio
###
from datetime import datetime
from math import floor, ceil
import time
from time import sleep
###
import json
###
from dotenv import load_dotenv
load_dotenv()
###
import zmq
zContext = zmq.asyncio.Context()
###
import logging
logger = logging.getLogger()
# from zmq.log.handlers import PUBHandler
# zmq_log_handler = PUBHandler('tcp://127.0.0.1:55356')
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} > {message}', style='{'))
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} #{lineno:>3} > {message}', style='{'), logging.DEBUG)
# zmq_log_handler.setRootTopic('greeter')
# logger.addHandler(zmq_log_handler)

logger.setLevel(logging.DEBUG)
###
from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
client = MongoClient()
db=client.fsmbot
###
import ccxt
exchange = getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})
###
from trade_machine import TradeModel, trades, load_trades

price_ticker = {}

import zmq
import zmq.asyncio
zContext = zmq.asyncio.Context()

In [2]:
async def run_ticker():
    socket = zContext.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        _, symbol, ticker = tickerString.split()
        ticker = json.loads(ticker)
        price_ticker[symbol] = ticker
        if symbol in trades:        
            for model in trades[symbol].values():
                if model.state in ['LIVE', 'LONG']:
                    model.tick(ticker=ticker)

from threading import Thread

ticker_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

ticker_thread = Thread(target=setup_loop, args=(ticker_loop,))
ticker_thread.start()
ticker_future = asyncio.run_coroutine_threadsafe(run_ticker(), ticker_loop)  

load_trades_from_tb()

In [7]:
# price_ticker['ALGOUSDT']['curDayClose']

logger.setLevel(logging.INFO)
load_trades(exchange, db.trades)
trades

INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'


{'LUNAUSDT': {'617463b16ffd55564f8552bd': 617463b16ffd55564f8552bd LONG LUNAUSDT 480.7692307692279},
 'EGLDUSDT': {'61748c806ffd55564f8552c0': 61748c806ffd55564f8552c0 LONG EGLDUSDT DotMap()}}

In [6]:
# ******************************************************
# *****************  !!! LIVE !!!  *********************
# ******************************************************
# (capital, risk) = 500, 0.005
symbol, state = 'EGLDUSDT', 'LONG'
# trade = TradeModel(exchange, db.trades)
# trade.insert(data={"state": state, "symbol" : symbol, "stop": 264.0, "target": 300.0, "availableQty": 83.6, 'curStopResp': {'orderId': '413774940'}})
# trade.start()
# trade.watch()

trade = trades[symbol]['61748c806ffd55564f8552c0']
# trade

# trade.update(data={'$set' : {'state': 'LONG', 'trigger': 273.80}})
# trade.setstop(data={'$set' : {'stop' : 265.70}})
# trade.movestop(data={'$set' : {'stop' : 265.60}})

trade.update(data={'$set' : {'state': 'LONG', 'trigger': 273.80}})
# qty, size

INFO:transitions.core:Executed callback 'saverec'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'remove_stop_on_exchange'
INFO:transitions.core:Executed callback 'set_stop_on_exchange'
INFO:transitions.core:Finished processing state LONG exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


True

ERROR:root:binance Account has insufficient balance for requested action.
INFO:transitions.core:Executed callback 'buy_on_exchange'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'set_stop_on_exchange'
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


In [8]:
# ACTIVE
(capital, risk) = 50000, 0.005
symbol, state = 'ALGOUSDT', 'DISABLED'
(trigger, stop, target) = 1.8831, 1.8750, 1.9396
qty = (capital * risk) / (trigger - stop) 
size = qty * trigger
# trade = TradeModel(exchange, db.trades)
# trade.insert(data={"state": state, "symbol" : symbol, "trigger": trigger, "stop": stop, "target": target, "qty": qty})
# trade.start()
# trade.watch()

# trade = trades[symbol]['617468356ffd55564f8552be']
# trade
# /trade = trade
# trade.sell()
# trade.movestop(data={'$set' : {'stop' : 1.873}})
qty, size


(30864.197530864214, 58120.3703703704)

In [33]:
# ACTIVE
(capital, risk) = 50000, 0.01
symbol, state = 'LUNAUSDT', 'DISABLED'
(trigger, stop, target) = 42.34, 41.30, 45
# qty = (capital * risk) / (trigger - stop) 
# size = qty * trigger
# trade = TradeModel(exchange, db.trades)
# trade.insert(data={"state": state, "symbol" : symbol, "trigger": trigger, "stop": stop, "target": target, "qty": qty})
# trade.start()
# trade.watch()

trade = trades[symbol]['617463b16ffd55564f8552bd']
# trade
# /trade = trade
# trade.sell()
trade.movestop(data={'$set' : {'stop' : 41.87}})
# qty, size

# 437.3 * 1.002
# ****************************************************************************************************************
# ****************************************************************************************************************


INFO:transitions.core:Executed callback 'saverec'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'remove_stop_on_exchange'
INFO:transitions.core:Executed callback 'set_stop_on_exchange'
INFO:transitions.core:Finished processing state LONG exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


True

In [11]:
# ACTIVE
(capital, risk) = 50000, 0.005
symbol, state = 'ATOMUSDT', 'DISABLED'
(trigger, stop, target) = 35.60, 35.34, 37.13
qty = (capital * risk) / (trigger - stop) 
size = qty * trigger
trade = TradeModel(exchange, db.trades)
trade.insert(data={"state": state, "symbol" : symbol, "trigger": trigger, "stop": stop, "target": target, "qty": qty})
trade.start()
trade.watch()
qty, size

INFO:transitions.core:Executed callback 'insertrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Finished processing state DISABLED exit callbacks.
INFO:transitions.core:Finished processing state LIVE enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


(961.5384615384689, 34230.769230769496)

INFO:transitions.core:Executed callback 'buy_on_exchange'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'set_stop_on_exchange'
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


In [5]:
# exchange.fetchOpenOrders('AVAXUSDT')
load_trades(exchange, db.trades)
trades

{'DOTUSDT': {'616067775fc1d120b18fb21c': 616067775fc1d120b18fb21c LONG DOTUSDT 602.4096385542181},
 'ALGOUSDT': {'6160ad5cd6fc93a6a65debb0': 6160ad5cd6fc93a6a65debb0 LONG ALGOUSDT 11961.722488038324}}

In [18]:
# # trades
# for symbol, trade_chunk_arr in trades.items():
#     for chunk in trade_chunk_arr:
#         model = chunk['model'] 
#         machine = chunk['machine']
#         if model.state != 'LONG':
#             continue
#         trade = model.trade
#         stop_order = exchange.fetchOrder(trade['curStopResp']['id'], symbol=symbol)
#         if exchange.fetchOrderStatus(trade['curStopResp']['id'], symbol='ALGOUSDT') == 'closed':
#             print(trade['id'], 'closed')
#         # print(chunk)
#     # print(tradeChunk)
# # tradeModel.trade

In [21]:
# logger.setLevel(logging.INFO)
trade.state, trade.rec.symbol

('LIVE', 'BNBUSDT')